In [ ]:
# データセット確認
from huggingface_hub import list_datasets

all_datasets = list_datasets() # ジェネレータ
for i in range(10):
  print(next(all_datasets).id)

In [ ]:
# 方法: v4.0.0 以降の datasets では、データセット内のコード(例：wrime.py)実行がサポートされていないため、3.6.0 に落とす
## 参考PR: https://github.com/huggingface/datasets/pull/7592
## 参考issue: https://github.com/huggingface/datasets/issues/7693
!pip uninstall -y datasets
!pip install datasets==3.6.0

In [ ]:
# データセット読み込み
from datasets import load_dataset

## 日本語データセットを利用( https://huggingface.co/datasets/shunk031/wrime より)
ja_emotions = load_dataset("shunk031/wrime", name="ver1")
ja_emotions ## DatasetDict(train: 40000件, val: 1200件, test: 2000件)

In [ ]:
# train データセット確認
train_ds = ja_emotions["train"]
print(len(train_ds)) # 40000

print(train_ds.column_names) # sentence, user_id, datetime, writer, reader1, reader2, reader3, avg_readers
print(train_ds[0])

print(train_ds.features)

In [ ]:
# train データセット確認(複数)
print(train_ds[:5]) # 冒頭5つ、各列がリスト形式
print(train_ds["sentence"][:5]) # 冒頭5つの文章のみ

In [ ]:
# web からのデータダウンロード例
load_dataset("csv", data_files="https://huggingface.co/datasets/transformersbook/emotion-train-split/raw/main/train.txt", sep=";", names=["text", "label"])

In [ ]:
# DataFrame への出力変更
import pandas as pd

# DatasetDict の出力形式変更
ja_emotions.set_format(type="pandas")
df = ja_emotions["train"][:]
df.head()

# # Dataset はそのままで OK
# df = pd.DataFrame(train_ds[:5])
# df

In [ ]:
# DataFrame に感情列を追加
def avg_readers2emotion(row):
  max = -1
  for emotion, label in row.items():
    if label > max:
      row_emotion = emotion
      max = label
  return row_emotion

df["emotion"] = df["avg_readers"].apply(avg_readers2emotion)
df.head()

In [ ]:
# クラス分布描画
import matplotlib.pyplot as plt

df["emotion"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes") # 日本語は文字化けしないよう設定が必要
plt.show()

In [ ]:
# クラスごとのテキスト長描画
df["Chars Per Sentence"] = df["sentence"].str.len() # 文字数で計算
df.boxplot("Chars Per Sentence", by="emotion", grid=False, showfliers=False, color="black") # 箱ひげ図
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
# 出力形式リセット
ja_emotions.reset_format()